In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [17]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [26]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = dict((c, i) for i, c in enumerate(world_set))

In [27]:
print(vocab)

{'u': 0, 'o': 1, "'": 2, ' ': 3, 'p': 4, 'f': 5, 'k': 6, 'a': 7, 'd': 8, 'm': 9, 'y': 10, 'i': 11, 'h': 12, 'c': 13, 'w': 14, 't': 15, 'n': 16, 'l': 17, ',': 18, 'B': 19, 'r': 20, 'e': 21, 'b': 22, 'g': 23, '.': 24, 's': 25}


In [28]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [29]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 13  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.03

In [31]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick walls a -> rick walls ar
1 rick walls ar -> ick walls are
2 ick walls are -> ck walls are 
3 ck walls are  -> k walls are t
4 k walls are t ->  walls are th
5  walls are th -> walls are the
6 walls are the -> alls are ther
7 alls are ther -> lls are there
8 lls are there -> ls are there 
9 ls are there  -> s are there f
10 s are there f ->  are there fo
11  are there fo -> are there for
12 are there for -> re there for 
13 re there for  -> e there for a
14 e there for a ->  there for a 
15  there for a  -> there for a r
16 there for a r -> here for a re
17 here for a re -> ere for a rea
18 ere for a rea -> re for a reas
19 re for a reas -> e for a reaso
20 e for a reaso ->  for a reason
21  for a reason -> for a reason 
22 for a reason  -> or a reason a
23 or a reason a -> r a reason an
24 r a reason an ->  a reason and
25  a reason and -> a reason and 
26 a reason and  ->  reason and y
27  reason and y -> reason and yo
28 reason and yo -> eason and you
29 eason and you -> ason

In [32]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[19, 20, 11, 13, 6, 3, 14, 7, 17, 17, 25, 3, 7]
[20, 11, 13, 6, 3, 14, 7, 17, 17, 25, 3, 7, 20]


In [35]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [37]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([185, 13, 26])
레이블의 크기 : torch.Size([185, 13])


In [40]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [41]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([20, 11, 13,  6,  3, 14,  7, 17, 17, 25,  3,  7, 20])


In [42]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
      x, _status = self.rnn(x)
      x = self.fc(x)
      return x

In [43]:
net = Net(vocab_size, hidden_size, 2)

In [44]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [45]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([185, 13, 26])


In [47]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

dapeededeeedephdpeddedeehhepphaaddeddededheehedhdhdepdddedeedpdpeddedheededeeedephddedpeddedphdedddeeedhdedaeeedpppdddhpdeeedededpdeedpdpeddedheededeeedephdpeddedphdhdhdeeededededeeedeededhddpdddea
e                                                                                                                                                                                                    
e                                                                                                                                                                                                    
                                                                                                                                                                                                     
     t     t   t     t        t       t        t   t  t  t  t t         t     t      t     t  t       t    t   t            t t   t  t t         t     t   t     t  t    t  t   t     t  t    t  t   
     t    

In [48]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [55]:
sentence = ("모기 피하려면 녹색 옷 입으세요"
            "모기에게 물리지 않으려면 모기가 좋아하는 붉은색이나 주황색 옷은 피하는 게 좋다"
            "녹색 옷을 입으면 모기에게 덜 물릴 수 있다")

In [4]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.36 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 66.7 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-08-16 13:32:23--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22e9:9f55, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bi

In [56]:
from konlpy.tag import Mecab
tokenizer = Mecab()
sentence_token=tokenizer.morphs(sentence)
sentence_set=list(set(sentence_token))

In [57]:
vocab = dict((c, i) for i, c in enumerate(sentence_set))
print(vocab)

{'아': 0, '는': 1, '하': 2, '피하': 3, '붉은색': 4, '다': 5, '으면': 6, '으려면': 7, '좋': 8, '가': 9, '수': 10, '덜': 11, '녹색': 12, '을': 13, '은': 14, '물릴': 15, '지': 16, '않': 17, '려면': 18, '주황색': 19, '이나': 20, '으세요': 21, '모기': 22, '옷': 23, '입': 24, '에게': 25, '물리': 26, '게': 27, '있': 28}


In [58]:
vocab_size = len(vocab)
print('형태소 집합 크기 : {}'.format(vocab_size))

형태소 집합 크기 : 29


In [59]:
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 12  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.03

In [61]:
# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence_token) - (sequence_length)):
  x_str = sentence_token[i:i + sequence_length]
  y_str = sentence_token[i + 1: i + sequence_length + 1]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['모기', '피하', '려면', '녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않'] -> ['피하', '려면', '녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면']
1 ['피하', '려면', '녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면'] -> ['려면', '녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기']
2 ['려면', '녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기'] -> ['녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가']
3 ['녹색', '옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가'] -> ['옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋']
4 ['옷', '입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋'] -> ['입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋', '아']
5 ['입', '으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋', '아'] -> ['으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋', '아', '하']
6 ['으세요', '모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋', '아', '하'] -> ['모기', '에게', '물리', '지', '않', '으려면', '모기', '가', '좋', '아', 

In [62]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[22, 3, 18, 12, 23, 24, 21, 22, 25, 26, 16, 17]
[3, 18, 12, 23, 24, 21, 22, 25, 26, 16, 17, 7]


In [63]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [64]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([29, 12, 29])
레이블의 크기 : torch.Size([29, 12])


In [78]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
  
  def forward(self, x):
      x, _status = self.rnn(x)
      x = self.fc(x)
      return x

In [79]:
net = Net(vocab_size, hidden_size, 2)

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [80]:
outputs = net(X)
print(outputs.shape)

torch.Size([29, 12, 29])


In [81]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1,vocab_size), Y.view(-1))
    #X shape : batchesxfeatures, output classes / Y : batches
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2) 
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([sentence_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += sentence_set[result[-1]]

    print(predict_str)

있피하있있피하있피하있있좋있피하있있있있피하있있있있있피하피하있있피하있있있피하있있피하않있피하있있있
좋좋모기모기모기모기모기모기모기모기모기모기모기모기모기모기모기모기수모기모기좋모기모기는모기모기모기모기모기모기모기모기모기모기모기모기모기모기모기
좋는좋옷는는는는는좋는좋모기는좋옷좋는는좋옷옷는좋는좋좋옷옷옷는는는모기는는좋는옷옷
좋좋좋옷는는모기모기물리좋는좋모기는좋옷좋는붉은색좋주황색옷는모기는좋좋옷좋옷는는는모기모기옷좋는좋는
좋좋좋옷는는모기모기물리좋는좋모기는좋아하는붉은색좋주황색좋주황색좋는게좋다좋옷는는으면모기다물리옷는좋는
옷좋옷옷는는모기모기물리좋는으려면모기는좋아하는붉은색이나주황색옷으려면모기는게좋다녹색옷는입으면모기모기아하하는려면
옷옷옷하는으면모기에게물리지는으려면모기가좋아하는붉은색이나주황색옷으려면피하는게좋다녹색옷을피하으면모기에게물리옷으면에게물리
다옷아하는으면모기에게물리지는으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게물리지으면에게물리
다덜아하는으면모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게물리하수에게덜
에게덜녹색피하는으면모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게물리지수다덜
에게덜녹색옷는으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게물리지수이나덜
에게덜녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게덜물릴수에게덜
에게덜녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게덜물릴수에게다
에게덜녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게덜물릴수에게다
에게덜녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게덜물릴수있다
에게덜녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게덜물릴수있다
에게덜녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은

In [82]:
predict_str

'에게려면녹색옷입으세요모기에게물리지않으려면모기가좋아하는붉은색이나주황색옷은피하는게좋다녹색옷을입으면모기에게덜물릴수있다'

In [83]:
print(sentence)

모기 피하려면 녹색 옷 입으세요모기에게 물리지 않으려면 모기가 좋아하는 붉은색이나 주황색 옷은 피하는 게 좋다녹색 옷을 입으면 모기에게 덜 물릴 수 있다


맨 앞에 단어 몇개가 부정확하다